In [ ]:
"""Introduction to pytorch computation graph"""
from functools import wraps
from decorator import decorator
from collections.abc import Iterable, Callable

import nographs as nog
from graphviz import Digraph

import torch
# import torch.nn as nn
# import torch.nn.functional as F
from torchvision import models

In [ ]:
nn_model = models.convnext_tiny()
#nn_model

In [ ]:
node_style ={"style": 'filled',            "shape": 'box',
             "align": 'left',              "fontsize": '12',
             "ranksep":'0.01',             "height":'0.02',   "width":'0.04',}
gr_style = {"linelength": '16', "rankdir": 'LR'}#"landscape":'True',

def adapter_graphviz( wrapped ):
    """translation to GraphViz visitor"""
    def node_id( item ):
        return str(hash(item))
    dot = Digraph( node_attr = node_style, graph_attr = gr_style, )
    @wraps( wrapped )
    def wrapper_bg( _item, _trav ):
        dot.node( node_id(_item), f"{str(type(_item))[:20]}\n{str(_item)[:20]}" )
        for result in wrapped(_item, _trav):
            dot.edge( node_id(_item), node_id(result[0]), str(result[1])[:30], )
            yield result
    wrapper_bg.dot = dot
    return wrapper_bg

In [ ]:
important = {'children'}
stop_types = {str, torch.Tensor, int, bool, }
sure_not = set(dir(object))
printed_types = set()

In [ ]:
@adapter_graphviz
def all_attr(var, _):
    global printed_types
    print( f"0000000------ {str(var)[:30]:30} {str(type(var)):30}")
    if isinstance( var, dict):
        pass
        # for i,n in zip(item[0].items(),inge(200)):
        #     yield (i[1], f"dict[{i[0]}]"),True
    elif isinstance( var, Iterable) and type(var) != str:
        for i,n in zip(var,range(200)):
            yield (i,f"[{n}]")
    else:
        for attr_name, _ in zip(dir(var), range(2000)):
            if attr_name in sure_not:
                continue
            attr = getattr(var, attr_name)
            # if isinstance( attr, Iterable) and type(attr) != str and not isinstance( var, dict):
            #     for i,n in zip(attr,range(200)):
            #         yield (i,f"[{n}]")
            #     continue

            if isinstance( attr, Callable):
                try:
                    attr = attr()
                except:
                    try:
                        attr = attr.__name__
                        if attr == attr_name:
                            attr = '-'*15
                    except:
                        pass
            
            if attr_name in important:
                yield ( attr, attr_name )
            else:
                if type(var) not in printed_types:
                    printed_types.add(type(var))
                    try:
                        print( f"111----, {attr_name:20}  {str(attr)[:20]:20} {str(type(attr))[:20]}")
                    except:
                        print( f"111----, {attr_name:20}")
                        print('#'*300)

trav_forward = nog.TraversalBreadthFirst(next_labeled_edges=all_attr)
trav_forward.start_from( nn_model, build_paths=True )

for _, _ in zip(trav_forward, range(7000)):
    pass

In [ ]:
#print(all_attr.dot.source)              # pylint: disable=pointless-statement
all_attr.dot